In [3]:
import sys
import os
import collections
import h5py
import csv

from chemicalchecker.util import logged, get_parser, save_output, features_file
from chemicalchecker.database import Dataset, Molrepo

from cmapPy.pandasGEXpress import parse

In [4]:
def pertid_to_meta(map_files):
    filenames = [map_files["GSE92742_Broad_LINCS_pert_info"]]
    pertid_meta = collections.defaultdict(set)
    for filename in filenames:
        with open(filename, "r") as f:
            reader = csv.reader(f, delimiter = "\t")
            h = next(reader)
            pert_id_idx = h.index("pert_id")
            pert_iname_idx = h.index("pert_iname")
            pert_type_idx  = h.index("pert_type")
            for r in reader:
                pertid_meta[r[pert_id_idx]].update([(r[pert_iname_idx], r[pert_type_idx])])
    return pertid_meta

def parse_molrepo():
    lincs_inchikey = {}
    molrepos = Molrepo.get_by_molrepo_name("lincs")
    for molrepo in molrepos:
        if not molrepo.inchikey:
            continue
        lincs_inchikey[molrepo.src_id] = molrepo.inchikey
    return lincs_inchikey

which_datasources = ["touchstone_conn_SUMMLY"]

dataset_code = 'D1.003'

dataset = Dataset.get(dataset_code)

map_files = {}

# Data sources associated to this dataset are stored in map_files
# Keys are the datasources names and values the file paths.
# If no datasources are necessary, the list is just empty.
for ds in dataset.datasources:
    map_files[ds.datasource_name] = ds.data_path + "/" + ds.filename


2019-05-07 17:51:49,677 chemicalchecker.util.config.config.Config [DEBUG   ] Loading config from: /aloy/home/mduran/cc_config.json
2019-05-07 17:51:49,758 chemicalchecker.util.config.config.Config [DEBUG   ] Loading config from: /aloy/home/mduran/cc_config.json
2019-05-07 17:51:49,760 chemicalchecker.util.config.config.Config [DEBUG   ] Loading config from: /aloy/home/mduran/cc_config.json
2019-05-07 17:51:49,762 chemicalchecker.util.config.config.Config [DEBUG   ] Loading config from: /aloy/home/mduran/cc_config.json
2019-05-07 17:51:49,764 chemicalchecker.util.config.config.Config [DEBUG   ] Loading config from: /aloy/home/mduran/cc_config.json
2019-05-07 17:51:49,766 chemicalchecker.util.config.config.Config [DEBUG   ] Loading config from: /aloy/home/mduran/cc_config.json
2019-05-07 17:51:49,770 chemicalchecker.util.config.config.Config [DEBUG   ] Loading config from: /aloy/home/mduran/cc_config.json
2019-05-07 17:51:49,772 chemicalchecker.util.config.config.Config [DEBUG   ] Loadin

In [5]:
which_datasources

['touchstone_conn_SUMMLY']

In [6]:
for ds in which_datasources:
    fn = map_files[ds]
    S  = parse.parse(fn)


In [18]:
import numpy as np
V = np.array(S.data_df)
pairs = collections.defaultdict(list)
for i, row in tqdm(enumerate(np.array(S.row_metadata_df.index))):
    for j, col in enumerate(np.array(S.col_metadata_df.index)):
        if V[i,j] < 90:
            continue
        else:
            if V[i,j] < 95:
                v = 1
            else:
                v = 2
        if col not in lincs_inchikey: continue
        pairs[(lincs_inchikey[col], row)] += [v]
pairs = dict((k, np.max(v)) for k,v in pairs.iteritems())

8798it [02:41, 54.51it/s]


In [13]:
lincs_inchikey = parse_molrepo()

2019-05-07 17:59:41,595 chemicalchecker.util.config.config.Config [DEBUG   ] Loading config from: /aloy/home/mduran/cc_config.json


In [17]:
from tqdm import tqdm

In [21]:
key_pairs = pairs
key_raw = collections.defaultdict(list)
for k, v in key_pairs.iteritems():
    key_raw[str(k[0])] += [(str(k[1]), v)]
features = sorted(set([x[0] for v in key_raw.values() for x in v]))


In [23]:
for k,v in key_raw.iteritems():
    print len(v)

286
197
290
362
461
320
383
200
437
275
570
176
446
215
230
134
261
471
386
227
176
273
338
191
364
265
327
330
254
255
150
222
301
181
145
289
235
215
262
249
318
288
289
417
242
214
285
218
188
228
235
267
240
203
147
170
214
150
187
208
180
353
246
172
364
248
196
258
474
391
213
288
239
182
241
226
263
207
285
376
679
261
148
432
208
331
406
280
475
409
211
182
279
277
200
231
231
488
226
316
360
451
317
324
301
265
259
335
389
481
534
452
277
182
192
251
231
219
524
315
434
412
310
240
216
338
339
580
229
300
269
218
315
320
234
290
356
178
195
220
489
226
196
199
127
423
646
250
219
429
474
130
109
170
200
545
220
148
272
354
161
326
227
295
233
274
255
253
305
353
314
284
539
206
165
428
355
160
399
178
189
352
261
248
173
286
305
336
242
358
293
217
154
192
212
259
284
223
233
246
301
325
517
220
402
193
330
435
181
224
343
234
485
322
459
183
332
235
239
288
228
218
284
366
284
227
308
363
244
188
310
225
204
199
324
207
463
255
284
270
589
197
255
819
159
231
309
262
199
245


In [6]:
from chemicalchecker.core.sign0 import sign0
s0 = sign0(".", ".", "D1.003")

from chemicalchecker.core.sign1 import sign1
s1 = sign1(".", ".", "D1.003")

s1.fit(s0)


2019-05-08 16:55:19,636 chemicalchecker.core.signature_base.BaseSignature [INFO    ] Creating model_path in: /aloy/home/mduran/myscripts/chemical_checker/package/scripts/preprocess/D1.003/models
2019-05-08 16:55:19,639 chemicalchecker.core.signature_base.BaseSignature [INFO    ] Creating stats_path in: /aloy/home/mduran/myscripts/chemical_checker/package/scripts/preprocess/D1.003/stats
2019-05-08 16:55:19,641 chemicalchecker.core.sign0.sign0 [DEBUG   ] signature path is: .
2019-05-08 16:55:19,643 chemicalchecker.core.sign0.sign0 [DEBUG   ] data_path: /aloy/home/mduran/myscripts/chemical_checker/package/scripts/preprocess/D1.003/sign0.h5
2019-05-08 16:55:19,644 chemicalchecker.util.config.config.Config [DEBUG   ] Loading config from: /aloy/home/mduran/cc_config.json
2019-05-08 16:55:19,656 chemicalchecker.core.sign1.sign1 [DEBUG   ] signature path is: .
2019-05-08 16:55:19,657 chemicalchecker.core.sign1.sign1 [WARNING ] Sign1 using default value (True) for discrete parameter
2019-05-08 

2019-05-08 16:56:12,720 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:12,724 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:12,728 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:12,732 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:12,736 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:12,740 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:12,746 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:12,751 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:12,755 smart_open.smart_open_lib [WARNI

2019-05-08 16:56:13,045 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,049 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,054 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,060 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,066 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,070 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,075 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,081 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,086 smart_open.smart_open_lib [WARNI

2019-05-08 16:56:13,383 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,387 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,392 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,396 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,401 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,406 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,410 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,414 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,419 smart_open.smart_open_lib [WARNI

2019-05-08 16:56:13,725 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,730 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,735 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,740 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,745 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,750 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,755 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,760 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:13,765 smart_open.smart_open_lib [WARNI

...

2019-05-08 16:56:59,433 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,438 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,443 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,451 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,459 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,463 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,469 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,473 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,479 smart_open.smart_open_lib [WARNI

2019-05-08 16:56:59,831 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,837 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,846 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,851 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,856 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,860 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,864 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,869 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:56:59,875 smart_open.smart_open_lib [WARNI

2019-05-08 16:57:00,217 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,223 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,229 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,239 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,244 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,249 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,254 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,258 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,265 smart_open.smart_open_lib [WARNI

2019-05-08 16:57:00,596 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,604 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,609 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,615 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,621 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,631 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,638 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,643 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,648 smart_open.smart_open_lib [WARNI

2019-05-08 16:57:00,992 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:00,997 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:01,002 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:01,009 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:01,014 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:01,018 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:01,023 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:01,027 smart_open.smart_open_lib [WARNING ] this function is deprecated, use smart_open.open instead
2019-05-08 16:57:01,031 smart_open.smart_open_lib [WARNI

In [2]:
s1.validation_path

'/aloy/home/mduran/myscripts/chemical_checker/package/scripts/preprocess/D1.003'

In [3]:
s1.model_path

'/aloy/home/mduran/myscripts/chemical_checker/package/scripts/preprocess/D1.003/models'

In [4]:
s1.stats_path

'/aloy/home/mduran/myscripts/chemical_checker/package/scripts/preprocess/D1.003/stats'